In [1]:
import openai
import os

key = os.environ.get('OPENAI_API_KEY')
openai.api_key = key

In [ ]:
print(key)

In [4]:
response = openai.Completion.create(
    engine="text-davinci-002",
    prompt="What is the capital of France?",
    max_tokens=1024,
    n=1,
    stop=None,
    temperature=0.5,
)

generated_text = response["choices"][0]["text"]
print(generated_text)



The capital of France is Paris.


In [9]:
response = openai.Completion.create(
    engine="text-davinci-002",
    prompt="What is the capital of France? Respond with only the capital. If the provided place has no captial, reply with 'None'",
    max_tokens=1024,
    n=1,
    stop=None,
    temperature=0.5,
)

generated_text = response["choices"][0]["text"]
print(generated_text)



Paris
